# Starter for Cores

In this notebook we provide an explanation of the different choices for Cores. For each of them we include: 
- a written description of the Core
- a code demo how to use the Core

---

### The model has two main parts

The baseline CNN model (which is mostly based on [this work](https://openreview.net/forum?id=Tp7kI90Htd)) is constructed from two main parts:
- **core**: the core aims to (nonlinearly) extract features that are common between neurons. That is, we assume there exist a set of features that all neurons use but combine them in their own unique way.
- **readout**: once the core extracts the feautures, then a neuron reads out from those features by simply linearly combining those features into a single value. Finally, by passing this single value through a final nonlinarity (in this case `ELU() + 1`) we make sure that the model output is positive and we get the inferred firing rate of the neuron.

### Currently (March 2024) there are 4 versions of Core modules

1. Stacked2dCore
2. RotationEquivariant2dCore
3. TransferLearningCore
4. SE2dCore

Descriptions of each **Core** module can be found in the neuralpredictors/layers/cores/conv2d.py. However, for convenience we will include descriptions in each dedicated block. 

---

Before learning how to use the cores, let's load the data. Refer to notebook **0_baseline_CNN.ipynb** for more explanations.

### Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from nnfabrik.builder import get_data

device = "cpu"
random_seed = 42


### Instantiate DataLoader

In [2]:
# loading the SENSORIUM+ dataset
filenames = ['/project/data/static22846-10-16-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': True,
                 'batch_size': 32,
                 'scale':1,
                 'cuda': False
                 }

dataloaders = get_data(dataset_fn, dataset_config)

### Access the input images in a batch of train data

In [3]:
dl = dataloaders['train']['22846-10-16']

In [4]:
batch = next(iter(dl))
images = batch.images

In [5]:
images.shape

torch.Size([32, 1, 144, 256])

---

Next code cell contains configuration parameters for the core. We will explore what they mean after introducing available choices for core. 

In [12]:
model_config = {
    # core args
    'input_channels': 1,
    'input_kern': 9,
    'hidden_kern': 7,
    'hidden_channels': 64,
    'layers': 4,
    'depth_separable': True,
    'stack': -1,
    'gamma_input': 6.3831,
    'pad_input': True
}

### Stacked2dCore

Made up of layers layers of nn.sequential modules.
Allows for the flexible implementations of many different architectures, such as convolutional layers,
or self-attention layers.

In [11]:
from neuralpredictors.layers.cores import Stacked2dCore 
model_stacked2dcore = Stacked2dCore(**model_config)
stacked2dcore_out = model_stacked2dcore(images)
print(stacked2dcore_out.shape)

torch.Size([32, 64, 136, 248])


### RotationEquivariant2dCore 

A core built of 2d rotation-equivariant layers. For more info refer to https://openreview.net/forum?id=H1fU8iAqKX.

In [13]:
from neuralpredictors.layers.cores import RotationEquivariant2dCore
model_rotationequivariant = RotationEquivariant2dCore(**model_config)
rotationequvariant_out = model_rotationequivariant(images)

### SE2dCore

An extension of the Stacked2dCore class. The convolutional layers can be set to be either depth-separable
(as used in the popular MobileNets) or based on self-attention (as used in Transformer networks).
Additionally, a SqueezeAndExcitation layer (also called SE-block) can be added after each layer or the n final layers. Finally, it is also possible to make this core fully linear, by disabling all nonlinearities.
This makes it effectively possible to turn a core+readout CNN into a LNP-model.

In [14]:
from neuralpredictors.layers.cores import SE2dCore
model_se2dcore = SE2dCore(**model_config)
se2dcore_out = model_se2dcore(images)

### TransferLearningCore

 Core based on popular image recognition networks from torchvision such as VGG or AlexNet. Can be instantiated as random or pretrained. Core is frozen by default, which can be changed with the fine_tune argument.

Try loading pretrained VGG16 and AlexNet

In [15]:
from neuralpredictors.layers.cores import TransferLearningCore
model_transfer_learning = TransferLearningCore(tl_model_name='vgg16', **model_config)
vgg16_core_out = model_transfer_learning(images)

In [17]:
from neuralpredictors.layers.cores import TransferLearningCore
model_transfer_learning = TransferLearningCore(tl_model_name = 'alexnet', **model_config)
alexnet_core_out = model_transfer_learning(images)

---

We have created instances of core models, such as **model_stacked2dcore, vgg16_core**. Now we explore how the main configuration parameters (five parameters to be exact) for the model influence our input images and the core output.

In [84]:
model_config = {
    # core args
    'input_channels': 1,
    'input_kern': 9,
    'hidden_kern': 7,
    'hidden_channels': 64,
    'layers': 3
}

(`input_channels` = 1) specifes the number of input channels. In the dataset we have loaded we have gray scale images, so the number of input channels is 1. If you have colored RGB images, you will need to set the input channels to 3. If you want your model to take into account also the behavioral parameters, such as pupil size and running speed, you accordingly increase the input channels to 5. 
(`hidden_channels` = 64) specifies the number of hidden channels. This is freely up to the user to define. If you want to have different sized hidden layers, you can pass on a list of length (`layers`) 

(`input_kern` = 9) sets the size of convolutional kernel at the first layer. 
(`hidden_kern` = 7) sets the size of convolutional kernel for all hidde layers. If you want to have different sized convolutional kernels, you can pass on a list of length (`layers`)

In [85]:
model_stacked2dcore = Stacked2dCore(**model_config)
stacked2dcore_out = model_stacked2dcore(images)
stacked2dcore_out.shape

torch.Size([32, 192, 144, 256])

We see that the output of the core has:
- **192** channels as specified by the `hidden_channels` * `layers`. This implies the core ouputs a stack of all hidden layers. If you want your model to output only the last hidden layer, then set the (`stack` = -1), which is another model configuration parameter. 
- a height of **144** and a width of **256** same as input images. This implies that the images are padded with zeros to keep the same input dimensions at the output. This is specified throught the (`pad_input` = True) model configuration parameter. 

---

### References
- Code for the [model function](https://github.com/sinzlab/sensorium/blob/8660c0c925b3944e723637db4725083f84ee28c3/sensorium/models/models.py#L17)
- Code for the [core Module](https://github.com/sinzlab/neuralpredictors/blob/0d3d793cc0e1f55ec61c5f9f7a98318b5241a2e9/neuralpredictors/layers/cores/conv2d.py#L27)